In [1]:
import time
import tqdm 
import pandas as pd 

from __init__ import * 

import torch
import torch.nn as nn 
import torch.optim as optim 

from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class args:
    seed = 42
    max_seq_length = 128
    learning_rate = 1e-5
    batch_size = 256
    dr_rate = 0.3
    hidden_dim = 312
    num_epochs = 200
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
def sentiment_score(x):
    '''
    input:
        score: range(1, 6)
    
    output:
        if x > 3.0 : return 1 
        elif x < 3.0 : return 0 
        elif x == 3.0 : return np.NaN
    '''
    if x >= 3.5 :
        return 1
    
    elif x < 3.5 :
        return 0

In [4]:
train_path = os.path.join(YELP_DIR, 'train.csv')
valid_path = os.path.join(YELP_DIR, 'valid.csv')
test_path = os.path.join(YELP_DIR, 'test.csv')

train = pd.read_csv(train_path, encoding='utf-8-sig')
valid = pd.read_csv(valid_path, encoding='utf-8-sig')
test = pd.read_csv(test_path, encoding='utf-8-sig')


bert_model = BertModel.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')
tokenizer = BertTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')

Some weights of the model checkpoint at huawei-noah/TinyBERT_General_4L_312D were not used when initializing BertModel: ['fit_denses.1.bias', 'cls.predictions.transform.dense.weight', 'fit_denses.4.weight', 'fit_denses.4.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'fit_denses.1.weight', 'cls.predictions.transform.dense.bias', 'fit_denses.0.weight', 'cls.predictions.transform.LayerNorm.weight', 'fit_denses.2.weight', 'fit_denses.3.weight', 'fit_denses.2.bias', 'fit_denses.3.bias', 'cls.predictions.transform.LayerNorm.bias', 'fit_denses.0.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing

In [5]:
bert_model.config

BertConfig {
  "_name_or_path": "huawei-noah/TinyBERT_General_4L_312D",
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "initializer_range": 0.02,
  "intermediate_size": 1200,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "pre_trained": "",
  "structure": [],
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [6]:
train.loc[:,'stars'] = train.loc[:, 'stars'].apply(sentiment_score)
valid.loc[:, 'stars'] = valid.loc[:, 'stars'].apply(sentiment_score)
test.loc[:, 'stars'] = test.loc[:, 'stars'].apply(sentiment_score)

train = train.dropna().reset_index(drop=True)
valid = valid.dropna().reset_index(drop=True)
test = test.dropna().reset_index(drop=True)

In [8]:
class BERTDataset(Dataset):
    def __init__(self, args, dataframe, tokenizer):
        self.tokenizer = tokenizer 
        self.data = dataframe 
        self.reviews = dataframe.text 
        self.labels = dataframe.stars
        self.max_seq_length = args.max_seq_length

        

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):

        review = self.reviews[idx]

        inputs = self.tokenizer.encode_plus(
            review, 
            None,
            add_special_tokens=True, 
            max_length=self.max_seq_length, 
            padding='max_length', 
            return_token_type_ids=True, 
            truncation=True
        )

        input_ids = inputs['input_ids']
        masks = inputs['attention_mask']
        token_type_ids = inputs['token_type_ids']

        return (
            torch.tensor(input_ids, dtype=torch.long), # token_ids
            torch.tensor(masks, dtype=torch.long), # attention_mask
            torch.tensor(token_type_ids, dtype=torch.long), # token_type_ids
            torch.tensor(self.labels[idx], dtype = float) # labels
        )


class BERTClassifier(nn.Module):
    def __init__(self, args, bertmodel):
        super(BERTClassifier, self).__init__()
        self.bert = bertmodel
        self.classifier = nn.Linear(args.hidden_dim, 1)
        self.dropout = nn.Dropout(args.dr_rate)

    
    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert(input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids)
        pooler = output['pooler_output']
        pooler = self.dropout(pooler)
        fc_layer = self.classifier(pooler)
        return fc_layer 

def calc_accuracy(pred_y, true_y):
    pred_y = torch.sigmoid(pred_y)
    return ((pred_y > 0.5) == true_y).sum().detach().cpu().item()

In [9]:
set_seed(args)

train_dataset = BERTDataset(args, train, tokenizer)
valid_dataset = BERTDataset(args, valid, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=4)
valid_dataloader = DataLoader(valid_dataset, batch_size=args.batch_size, num_workers=4)


model = BERTClassifier(args, bert_model)
model.to(args.device)


criterion = nn.BCEWithLogitsLoss().to(args.device)
optimizer =optim.Adam(model.parameters(), lr = args.learning_rate, weight_decay = 0.001)

In [10]:
train_loss_list = []
train_acc_list = []
test_loss_list = []
test_acc_list = []

best_loss = float('inf')
set_seed(args)
for epoch in range(1, args.num_epochs+1):
    train_loss = 0 
    train_acc = 0

    test_loss = 0
    test_acc = 0 

    model.train()
    start_time = time.time()
    for batch in tqdm.tqdm(train_dataloader, desc = 'training...'):
        batch = tuple(t.to(args.device) for t in batch)

        inputs = {'input_ids':      batch[0], 
                  'attention_mask': batch[1], 
                  'token_type_ids':     batch[2]}
        label = batch[3]

        pred_y = model(**inputs).squeeze()
        loss = criterion(pred_y, label)

        train_acc += calc_accuracy(pred_y, label)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_acc /= len(train_dataset)
    train_loss /= len(train_dataset)
    train_acc_list.append(train_acc)
    train_loss_list.append(train_loss)

    model.eval()
    with torch.no_grad():
        for batch in tqdm.tqdm(valid_dataloader, desc = 'evaluating...'):
            batch = tuple(t.to(args.device) for t in batch)

            inputs = {'input_ids':      batch[0], 
                      'attention_mask': batch[1], 
                      'token_type_ids':     batch[2]}
            label = batch[3]

            pred_y = model(**inputs).squeeze()
            loss = criterion(pred_y, label)

            test_acc += calc_accuracy(pred_y, label)
            test_loss += loss.item()

    test_acc /= len(valid_dataset)
    test_loss /= len(valid_dataset)
    test_acc_list.append(test_acc)
    test_loss_list.append(test_loss)


    end_time = time.time()
    elapsed_mins, elapsed_secs = epoch_time(start_time, end_time)
    print(f'epoch [{epoch}/{args.num_epochs}], elapsed time: {elapsed_mins}m, {elapsed_secs:.2f}s')
    print(f'train loss: {train_loss:.4f}\ttrain accuracy: {train_acc*100:.2f}%')
    print(f'test loss: {test_loss:.4f}\ttest accuracy: {test_acc*100:.2f}%\n')

    if best_loss > test_loss :
        best_loss = test_loss 
        bert_param_path = os.path.join(BASE_DIR, 'baseline_parameters')
        if not os.path.exists(bert_param_path):
            os.makedirs(bert_param_path)

        torch.save(model.state_dict(), os.path.join(bert_param_path, f'bert_parameters.pt'))
    
df = pd.DataFrame([train_loss_list, test_loss_list, train_acc_list, test_acc_list], index = ['train_loss', 'test_loss', 'train_acc', 'test_acc']).T 


save_path = os.path.join(BASE_DIR, 'baseline')
if not os.path.exists(save_path):
    os.makedirs(save_path)

df.to_csv(f'{save_path}/bert_results.csv', encoding='utf-8-sig', index=False)

evaluating...: 100%|██████████| 378/378 [00:36<00:00, 10.23it/s]


epoch [1/200], elapsed time: 7m, 30.12s
train loss: 0.0013	train accuracy: 84.26%
test loss: 0.0012	test accuracy: 86.94%



evaluating...: 100%|██████████| 378/378 [00:36<00:00, 10.33it/s]


epoch [2/200], elapsed time: 7m, 31.31s
train loss: 0.0012	train accuracy: 86.78%
test loss: 0.0011	test accuracy: 87.55%



evaluating...: 100%|██████████| 378/378 [00:37<00:00, 10.16it/s]


...

epoch [198/200], elapsed time: 7m, 20.43s
train loss: 0.0003	train accuracy: 96.98%
test loss: 0.0027	test accuracy: 83.51%



evaluating...: 100%|██████████| 378/378 [00:37<00:00, 10.04it/s]


epoch [199/200], elapsed time: 7m, 18.78s
train loss: 0.0003	train accuracy: 97.00%
test loss: 0.0028	test accuracy: 83.21%



evaluating...: 100%|██████████| 378/378 [00:36<00:00, 10.35it/s]

epoch [200/200], elapsed time: 7m, 17.45s
train loss: 0.0003	train accuracy: 97.04%
test loss: 0.0028	test accuracy: 83.29%

